# Spotify Visualisation and Prediction
Welcome to my Spotify notebook, where today we will be visualising this dataset and creating a regressor to predict which year the songs were made.

<img src="https://th.bing.com/th/id/OIP.2mczJR1u19WDWERsDFrgNwHaFb?pid=Api&rs=1" width="200px"/>
<img src="https://www.dailydot.com/wp-content/uploads/d62/72/b41e5005a226e2ed.png" width="200px"/>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import boxcox
from collections import Counter
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/spotify-dataset-19212020-160k-tracks/data.csv')
df = df.drop('id', axis=1)

In [ ]:
df.head()

In [ ]:
artists = []
for i in df['artists']:
    for j in i.split(", "):
        artist = j.replace('[', '').replace(']', '')
        if artist[0] == "'":
            artist = artist[1:]
        if artist[-1] == "'":
            artist = artist[:-1]
        artists.append(artist)

# Most common artists
The first visualisation that we will be showing is a bar chart which shows the 100 most common music artists. As seen below, the most common artist in our dataset is Francisco Canaro with 2,228 songs. Some famous performers on our list include Chopin, Beethoven, Mozart, Sinatra, Johhny Cash, Bob Dylan, and The Beatles.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
count = pd.Series(Counter(artists)).sort_values(ascending=False)[:100]
bars = sns.barplot(count.keys(), count, palette='twilight')
for index in range(0, len(bars.patches), 2):
    bar = bars.patches[index]
    bars.annotate(format(bar.get_height(), '.0f'), (bar.get_x()+bar.get_width()/2., 
                                                    bar.get_height()), ha='center',
                 va='bottom', size=8)
    
plt.title('Most common artists')
plt.xlabel('Artists')
plt.ylabel('Number of artists')
plt.xticks(rotation=90, size=8)
plt.show()

# Features over time

Next, we plot out some line graphs which show the average values per year in the features. Here we see some interesting observations. First of all, we can conclude that the **acousticness** has plummeted over the years; going from around 90% in the 50s to 25% in the last few years. Also, the **danceability** has had a steady increase over time, with it being slightly lower than 50% in the 50s to roughly 60% in recent years.

Similarly, the loudness has climbed over the years from -16 to -8. Additionally, music seems to have gotten a lot more explicit in the 2000s, due to it having a value of 0.0 in the 50s while in the 2000s it reached around 0.17. The valence (happiness) of the songs has been on a rollercoaster, with the 70s and 80s producing the happiest songs, while the 2010s having a sudden drop.

In [ ]:
groups = df[df['year']>=1950].groupby(['year']).mean()
fig, axes1 = plt.subplots(4, 3, figsize=(17, 25))
plt.subplots_adjust(hspace=0.3, wspace=0.25)
fig, axes2 = plt.subplots(1, 2, figsize=(9, 4))
plt.subplots_adjust(wspace=0.25)

for row in axes1:
    for ax in row:
        col = groups.columns[list(axes1.flatten()).index(ax)]
        ax.plot(groups.index, groups[col])
        ax.set_title(col + ' over years', size=15)
        ax.set_xlabel('year')
        ax.set_ylabel(col)
        
for ax in axes2:
    col = groups.columns[list(axes2.flatten()).index(ax)+12]
    ax.plot(groups.index, groups[col])
    ax.set_title(col + ' over years', size=12)
    ax.set_xlabel('year')
    ax.set_ylabel(col)

plt.show()

# Heatmap
The subsequent heatmap shows us that the most correlated features are energy and loudness (0.78), followed by energy & year and danceability & valence (0.54).

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.heatmap(df.corr(), mask=np.triu(df.corr()), annot=True)
plt.title('Correlation of features', size=20)
plt.show()

# Scatterplot
Furthermore, we will now use a scatterplot to take a look at two of the most correlated pairs of features.

We have chosen "energy and loudness" and "year and popularity". As seen with **"energy and loudness"**, there is a slow and steady increase in the trend of our features. With the **"years and popularity"**, there is a lot more outliers and the data is more messy, however we can still make some solid conclusions from the plot. Along with scattering the data points, we have also created a line which (somewhat) accurately plots the trend of our samples.

In [ ]:
features_list = [['energy', 'loudness'], ['year', 'popularity']]
orders = [5, 1]
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for ax in axes:
    index = list(axes).index(ax)
    features = features_list[index]
    x = features[0]
    y = features[1]
    
    sns.regplot(df[x], df[y], line_kws={'color':'black'}, 
                order=orders[index], ax=ax)
    ax.set_xlabel(x)
    ax.set_ylabel(y)
    ax.set_title(x + " and " + y)

plt.suptitle('Correlation', size=20)
plt.show()

# Transformations
Additionally, we will now use the log transformation and box cox methods to transform our data so that its distribution resembles more of a bell shape.

We visualise this by going through every variable, plotting a histogram of the distribution and a boxplot below it.

In [ ]:
i = 0
for col in df.drop(['artists', 'name', 'explicit', 'mode', 'release_date'], axis=1).columns:
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    fig.tight_layout(h_pad=3)
    i += 1
    
    if i%3==0:
        colour1='lightgreen'
        colour2='blue'
    elif i%2==0:
        colour1='pink'
        colour2='green'
    else:
        colour1='skyblue'
        colour2='red'
    
    if col=='loudness':
        log = (df[col]+61).transform(np.log)
        box_cox = pd.Series(boxcox(df[col]+61)[0])
    else:
        log = (df[col]+1).transform(np.log)
        box_cox = pd.Series(boxcox(df[col]+1)[0])
    
    df[col].hist(ax=axes[0][0], color=colour1)
    log.hist(ax=axes[0][1], color=colour1)
    box_cox.hist(ax=axes[0][2], color=colour1)
    
    for transform in [[df[col], axes[0][0]], [log, axes[0][1]], [box_cox, axes[0][2]]]:
        ax = transform[1]
        deciles = pd.Series(transform[0]).quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])
        for pos in np.array(deciles).reshape(1, -1)[0]:
            handle = ax.axvline(pos, color='darkblue', linewidth=1)
        ax.legend([handle], ['deciles'])
    
    sns.boxplot(df[col], orient="h", ax=axes[1][0], color=colour2)
    sns.boxplot(log, orient="h", ax=axes[1][1], color=colour2)
    sns.boxplot(box_cox, orient="h", ax=axes[1][2], color=colour2)
    
    axes[1][0].set_xlabel('normal', size=15)
    axes[1][1].set_xlabel('log transform', size=15)
    axes[1][2].set_xlabel('box cox', size=15)
    axes[0][1].set_title(col, size=25, pad=15)
    
    plt.show()

# Binning
Now we will bin the data by compressing the unique amount of values per feature into twenty. The distribution per variable is shown below through histograms.

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(15, 20))
plt.subplots_adjust(hspace=0.3, wspace=0.27)
colours = ['skyblue', 'pink', 'lightgreen']*4

for row in axes:
    for ax in row:
        index = list(axes.flatten()).index(ax)
        col = df.drop(['artists', 'name', 'explicit', 'mode', 'release_date', 'year'], 
                      axis=1).columns[index]
        feature = df[col]
        count = Counter(np.digitize(feature, np.arange(feature.min(), feature.max(),
                                                            (feature.max()-feature.min())/20)))
        ax.bar(count.keys(), count.values(), color=colours[index])
        ax.set_title(col + ' bin')
        ax.set_xlabel('bins')
        ax.set_ylabel(col)

plt.show()

In [ ]:
for col in df[df.drop(['artists', 'name', 'explicit', 'mode', 'release_date'], 
                      axis=1).columns[3:12]]:
    if col=='loudness':
        df[col] = pd.Series(boxcox(df[col]+61)[0])
    else:
        df[col] = pd.Series(boxcox(df[col]+1)[0])

# Creating X and y
The final part of this notebook will be predicting the data. Our goal will be to predict the "year" feature using every other numerical variable.

The "X" is the features in our dataset except "year", while the "y" will be assigned to "year". Afterwards, we split the data further off into train and test sets.

In [ ]:
X = df[['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 
        'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness',
        'tempo', 'valence']]
y = df['year']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Predicting the data
We finish off by creating an XGBRegressor model, training it on the train set and achieving a pleasing accuracy around 75%.

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)

print('XGBoost')
print('Model score:    ', model.score(X_test, y_test))
print('Cross val score:', cross_val_score(model, X_test, y_test).mean())

<img src="https://miro.medium.com/max/593/1*pfmeGgGM5sxmLBQ5IQfQew.png" width="500px"/>

## Thank you for reading this notebook.
## If you enjoyed this notebook and found it helpful, please give it an upvote and provide feedback, as it would help me make more of these.